In [26]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [27]:
def gettingUrl(url):

    driver = webdriver.Chrome("/usr/bin/chromedriver")  #replace the file path by the address of chromedriver in your system
    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    #this renders the JS code and stores all of this information in static HTMl code.
    return html

In [28]:
#extracting data from acs nano site
html = gettingUrl("https://pubs.acs.org/doi/10.1021/acsnano.9b06394")

/tmp/ipykernel_30147/3389595816.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver")  #replace the file path by the address of chromedriver in your system


In [29]:
# #to find out the details of the research paper 5
# soup = BeautifulSoup(html, 'lxml')
# paperNames = soup.find_all('span',class_ = 'NLM_article-title')
# for paper in paperNames:
#     print(paper.text)

In [30]:
# for paper in paperNames:
#     print(paper.text)

In [31]:
#finding out the number of times the research paper number 5 is mentioned in the text.
soup_new = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
a_tags = soup_new.find('div', class_="article_content-left ui-resizable")
count = a_tags.find_all('a',class_="ref5")
print(len(count))

# index = 0

# for i in count:
#     data  = i.text
#     print(i)
#     dataextract = re.findall('(\(5)',data)
#     if(dataextract):
#         # print(dataextract)
#         index = index + 1

# # print(index)
        



6


In [32]:
soup2 = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
links = soup2.find('div', class_="article_content-left ui-resizable")
paperLinksgoogleScholar=links.find_all('a',class_="google-scholar")  # creating a list containing the google scholar links for all the reference papers.

In [33]:
index=0
scholarLinks=[]
for link in paperLinksgoogleScholar:
    scholarLinks.append(link['href'])   #A new list containing the google scholar links for only the first three papers is created. 
    index=index+1

In [34]:
for link in scholarLinks:
    print(link)

http://scholar.google.com/scholar_lookup?hl=en&volume=9&publication_year=2015&pages=9451-9469&author=H.+Zhang&title=Ultrathin+Two-Dimensional+Nanomaterials&doi=10.1021%2Facsnano.5b05040
http://scholar.google.com/scholar_lookup?hl=en&volume=10&publication_year=2016&pages=8121-8123&author=A.+T.+S.+Weeauthor=M.+C.+Hersamauthor=M.+Chhowallaauthor=Y.+Gogotsi&title=An+Update+from+Flatland&doi=10.1021%2Facsnano.6b06087
http://scholar.google.com/scholar_lookup?hl=en&volume=2&publication_year=2017&pages=16098&author=B.+Anasoriauthor=M.+R.+Lukatskayaauthor=Y.+Gogotsi&title=2D+Metal+Carbides+and+Nitrides+%28MXenes%29+for+Energy+Storage&doi=10.1038%2Fnatrevmats.2016.98
http://scholar.google.com/scholar_lookup?hl=en&volume=97&publication_year=2019&pages=24-25&author=B.+C.+Seijo&title=2019%3A+The+Year+the+Periodic+Table+Gets+Its+Due&doi=10.1021%2Fcen-09701-feature8
http://scholar.google.com/scholar_lookup?hl=en&publication_year=2019&author=B.+Anasori&author=Y.+Gogotsi&title=2D+Metal+Carbides+and+Nit

In [35]:
# for link in paperLinks:
#     driver.get(link)
#     driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()    
    


In [36]:
html = gettingUrl(scholarLinks[1])
soup3 = BeautifulSoup(html,'lxml')  #creating an instance of the BeautifulSoup library
content = soup3.find('div', class_="gs_a")
authors = content.find_all('a')
# print(authors)
for author in authors:
    print(author.text)

/tmp/ipykernel_30147/3389595816.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver")  #replace the file path by the address of chromedriver in your system


ATS Wee
MC Hersam
M Chhowalla


In [37]:
#to find the publication house name of the referenced paper from google scholar.

name = content.text
# print(name)

if "science" in name:
    print("science")
elif 'Wiley' in name:
    print("wiley")
elif 'nature' in name:
    print("nature")
elif 'sciencedirect' in name:
    print("sciencedirect")
elif 'Springer' in name:
    print("Springer")
elif 'Elsevier' in name:
    print("Elsevier")
elif 'mdpi' in name:
    print("mdpi")
elif 'ACS Publications' in name:
    print("ACS Publications")
elif 'ieee' in name:
    print("ieee")
else:
    print("other")

ACS Publications


In [38]:
#finding out the publication house link of the paper from google scholar.
paperLink = soup3.find('h3',class_ = 'gs_rt')
print(paperLink.a['href'])

https://pubs.acs.org/doi/full/10.1021/acsnano.6b06087


In [40]:
driver = webdriver.Chrome("/usr/bin/chromedriver")
driver.get(scholarLinks[4])
time.sleep(5)
# driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()
driver.find_element(By.LINK_TEXT, "Cite").click()



# print(driver.current_url)
# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "gs_citr"))
#     )
#     element.click()
# except:
#         driver.quit()


/tmp/ipykernel_30147/3514407175.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver")


In [41]:
html = gettingUrl(driver.current_url)
# print(driver.current_url)
soup4 = BeautifulSoup(html, 'lxml')
paperName = soup4.find('div',class_ = 'gs_citr')
print(paperName.i.text)  #printing the name of the 5th paper.

/tmp/ipykernel_30147/3389595816.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver")  #replace the file path by the address of chromedriver in your system


2D metal carbides and nitrides (MXenes)


In [ ]:
# url=input("Enter the url of the website: ")
# name = re.findall('www\.(.*?)\.', url)
# print(name)

In [ ]:
# # finding out the publication house link of the paper from google scholar.
# paperLink = soup4.find('h3',class_ = 'gs_rt')
# print(paperLink.a['href'])